In [1]:
import gc
from all_stand_var import conv_dict, vent_cols3
from all_own_funct import memory_downscale,memory_upscale,evaluate
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
from seaborn import heatmap
from sklearn.metrics import roc_curve, accuracy_score, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix
from scipy.stats import kurtosis
from sklearn.metrics import roc_curve, accuracy_score, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix
import locale
import pickle
locale.setlocale(locale.LC_ALL, 'fr_FR')
output_folder = os.path.join(os.getcwd(), 'Results_LR_RF_v2','Table')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


C:\Users\berend\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\berend\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\berend\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\berend\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [4]:
vent_cols3 = ['pat_hosp_id', 'pat_bd', 'pat_datetime', 'AdmissionDate','DischargeDate', 'mon_rr', 'mon_hr', 'mon_sat',
              'mon_etco2', 'vent_m_fio2', 'vent_m_ppeak','vent_m_peep',
             'mon_ibp_mean','pat_weight_act','vent_m_rr', 'vent_m_tv_exp']
dtype_dict={'vent_cat': 'category','vent_machine':'category','vent_mode':'category'}
df_raw=pd.read_csv(r'data\sorted_bron_date.csv',delimiter=';',converters=conv_dict,usecols=vent_cols3,dtype=dtype_dict,parse_dates=['pat_bd','pat_datetime','AdmissionDate', 'DischargeDate'],na_values=['NULL','null', 'Null','nUll','nuLl','nulL'])



In [5]:
import importlib
import LR_build as pp
importlib.reload(pp)
df = pp.data_pp_function(df_raw,path=output_folder,timestep='12H')
#df=df.groupby('Admissionnumber',sort=False).fillna(method='ffill').fillna(method='bfill')
#f = open(os.path.join('./Results_LR_RF_v2/', 'processed_df.txt'), 'rb')
#df = pickle.load(f)
#f.close()




394
c:\Users\berend\Documents\Python_Scripts\Patient_Selection\LR_build.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pat_datetime_temp'] = pd.to_datetime(df['pat_datetime']).dt.date
c:\Users\berend\Documents\Python_Scripts\Patient_Selection\LR_build.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pat_bd'] = pd.to_datetime(df['pat_bd']).dt.date
c:\Users\berend\Documents\Python_Scripts\Patient_Selection\LR_build.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

In [ ]:
print(df.isna().sum())

In [15]:
from tableone import TableOne
from all_stand_var import rename_dict
#del rename_dict['Diagnose']

df_temp=df.reset_index(drop=True)
df_temp=df_temp.rename(columns=rename_dict)
print(df_temp.info())
columns= list(rename_dict.values())

mytable=TableOne(df_temp,columns=columns,groupby='Reintub',pval=True)
print(mytable.tabulate(tablefmt="grid"))
mytable.to_excel(os.path.join('./Results','Table_1_raw.xlsx'))

                                                   | 90.5        |           | 1 (0.0)      | 1 (0.0)      |             |           |
+----------------------------------------------------+-------------+-----------+--------------+--------------+-------------+-----------+
|                                                    | 90.6        |           | 2 (0.0)      | 2 (0.0)      |             |           |
+----------------------------------------------------+-------------+-----------+--------------+--------------+-------------+-----------+
|                                                    | 90.7        |           | 1 (0.0)      | 1 (0.0)      |             |           |
+----------------------------------------------------+-------------+-----------+--------------+--------------+-------------+-----------+
|                                                    | 90.8        |           | 3 (0.0)      | 2 (0.0)      | 1 (0.0)     |           |
+------------------------------------------

In [19]:
df=df.reset_index(drop=True)
def index_1hr(group):
    #grouped = pd.Grouper(key='pat_datetime', freq='1H')
    #group['idx_1hr']=group.groupby(grouped,sort=False).ngroup().add(1)
    #group['idx_1hr']=group['idx_1hr'].apply(str)
    group['Time gap (min)'] = group['pat_datetime'].max()-group['pat_datetime'].min()
    group['Time gap (min)'] = group['Time gap (min)'].dt.seconds.divide(60)
    return group
#df.reset_index(drop=True,inplace=True)
df=df.groupby('Adnum',sort=False,as_index=False).apply(index_1hr)
print((df['Adnum'].value_counts()))
#df[['mis']]=StandardScaler().fit_transform(df[['mis']])

def x_modelling(df):
    #df=df[['pat_weight_act','Age','Adnum','mis']]
    df=df.drop(['Extubation_date','pat_datetime'],axis=1)
    df=df.groupby(['Adnum'],sort=False).agg(['mean'])
    df.columns = df.columns.droplevel(1)
    df=df.reset_index().drop_duplicates().set_index('Adnum')
    df=df.fillna(method='ffill').fillna(method='bfill')
    
    return df
df_temp=x_modelling(df)
print(df_temp.info())


0      720
208    720
84     720
339    720
338    720
      ... 
306    240
47      65
76      28
284     10
383      1
Name: Adnum, Length: 327, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 327 entries, 104 to 246
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pat_weight_act  327 non-null    float32
 1   mon_etco2       327 non-null    float32
 2   mon_hr          327 non-null    float32
 3   mon_ibp_mean    327 non-null    float32
 4   mon_rr          327 non-null    float32
 5   mon_sat         327 non-null    float32
 6   vent_m_fio2     327 non-null    float32
 7   vent_m_peep     327 non-null    float32
 8   vent_m_ppeak    327 non-null    float32
 9   vent_m_rr       327 non-null    float32
 10  vent_m_tv_exp   327 non-null    float32
 11  Age             327 non-null    float32
 12  Reintub         327 non-null    int64  
 13  Time gap (min)  327 non-null    float64
dtypes: float

In [20]:

df_temp=df_temp.rename(columns=rename_dict)
print(df_temp.info())
columns= list(rename_dict.values())

mytable=TableOne(df_temp,columns=columns,groupby='Reintub',pval=True)
print(mytable.tabulate(tablefmt="grid"))
mytable.to_excel(os.path.join('./Results','Table_1_pat.xlsx'))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 327 entries, 104 to 246
Data columns (total 14 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Weight (kgs)                             327 non-null    float32
 1   End tidal CO2 (mmHg)                     327 non-null    float32
 2   Heart rate                               327 non-null    float32
 3   Invasive blood pressure (mmHg)           327 non-null    float32
 4   Respiratory rate                         327 non-null    float32
 5   Saturation (%)                           327 non-null    float32
 6   Inspiratory oxygen (%)                   327 non-null    float32
 7   Ventilator PEEP (mmHg)                   327 non-null    float32
 8   Ventilator peak pressure (mmHg)          327 non-null    float32
 9   Ventilator respiratory rate              327 non-null    float32
 10  Ventilator expiratory tidal volume (mL)  327 non

In [ ]:
from sklearn.preprocessing import StandardScaler
df=df.reset_index(drop=False)
df.info()
def index_1hr(group):
    group['mis']=(group['pat_datetime'].max()-group['pat_datetime'].min()).total_seconds()/60
    return group
#grouped = pd.Grouper(key='pat_datetime', freq='60min',label='left',convention='start')
#df['idx_1hr']=df.groupby(['Adnum','pat_hosp_id','AdmissionDate','DischargeDate'],sort=False,as_index=False).ngroup().add(1)
df=df.groupby('Adnum',sort=False,as_index=False).apply(index_1hr)
print((df['Adnum'].value_counts()))
df[['mis']]=StandardScaler().fit_transform(df[['mis']])
df=df.set_index(['pat_hosp_id','AdmissionDate','DischargeDate'])



In [ ]:

pat=pd.read_excel(r'Results\admissiondate_v2.xlsx', parse_dates=[
                            'AdmissionDate', 'DischargeDate'])
group=pat.groupby('pat_hosp_id',sort=False).max().reset_index()
group.drop_duplicates('pat_hosp_id',inplace=True)
df_train,df_val,df_test=pp.split_stratified_into_train_val_test(group, stratify_colname='Reintub',
                                         frac_train=0.60, frac_val=0.1, frac_test=0.3,
                                         random_state=1)
print(df_train.info())

train_pat=df_train['pat_hosp_id'].unique()
test_pat=df_test['pat_hosp_id'].unique()
val_pat=df_val['pat_hosp_id'].unique()

df_train = df[df.index.get_level_values('pat_hosp_id').isin(train_pat)]
df_val = df[df.index.get_level_values('pat_hosp_id').isin(val_pat)]
df_test = df[df.index.get_level_values('pat_hosp_id').isin(test_pat)]


In [ ]:

def y_modelling(df):
    y=df[['Reintub','Adnum']]
    y=y.groupby(['Adnum'],sort=False).agg(['max'])
    y.columns = ["_".join(a) for a in y.columns.to_flat_index()]
    y=y.reset_index().drop_duplicates().set_index('Adnum')
    y=y[~y.index.duplicated(keep='last')]
    y=y.fillna(method='ffill').fillna(method='bfill')
    return y
Y_TRAIN=y_modelling(df_train)
Y_TEST=y_modelling(df_test)
Y_VAL=y_modelling(df_val)


In [ ]:
print(Y_TRAIN.value_counts())
print(Y_VAL.value_counts())
print(Y_TEST.value_counts())
Y_TRAIN=Y_TRAIN['Reintub_max'].to_numpy()
Y_TEST=Y_TEST['Reintub_max'].to_numpy()
Y_VAL=Y_VAL['Reintub_max'].to_numpy()
Y_TRAIN=np.append(Y_TRAIN,Y_VAL)


In [ ]:
def x_modelling(df):
    #df=df[['pat_weight_act','Age','Adnum','mis']]
    df=df.drop(['Extubation_date','level_0','pat_datetime', 'Reintub'],axis=1)
    df=df.groupby(['Adnum'],sort=False).agg(['mean'])
    df.columns = ["_".join(a) for a in df.columns.to_flat_index()]
    df=df.reset_index().drop_duplicates().set_index('Adnum')
    df=df.fillna(method='ffill').fillna(method='bfill')
    return df
X_TRAIN=x_modelling(df_train)
X_TEST=x_modelling(df_test)
X_VAL=x_modelling(df_val)
print(X_TRAIN.info())

In [ ]:
X_TRAIN=X_TRAIN.append(X_VAL)

In [ ]:
f = open(os.path.join(output_folder, 'x_train.txt'), 'wb')
pickle.dump(X_TRAIN, f)
f.close()

f = open(os.path.join(output_folder, 'x_test.txt'), 'wb')
pickle.dump(X_TEST, f)
f.close()

f = open(os.path.join(output_folder, 'y_train.txt'), 'wb')
pickle.dump(Y_TRAIN, f)
f.close()

f = open(os.path.join(output_folder, 'y_test.txt'), 'wb')
pickle.dump(Y_TEST, f)
f.close()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 50, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [1,2,3,4, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 30, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_TRAIN, Y_TRAIN)





In [ ]:
from sklearn.metrics import classification_report, confusion_matrix,average_precision_score,f1_score,roc_curve,roc_auc_score,plot_confusion_matrix

base_model = RandomForestClassifier(n_estimators = 10, random_state = 42,max_depth=10)
base_model.fit(X_TRAIN, Y_TRAIN)
base_accuracy = evaluate(base_model, X_TEST, Y_TEST,'base RF',output_folder)

best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_TEST,Y_TEST,'Best random RF',output_folder)

if base_accuracy > random_accuracy:
    best_random = base_model


In [ ]:
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix,average_precision_score,f1_score,roc_curve,roc_auc_score,plot_confusion_matrix
from matplotlib.backends.backend_pdf import PdfPages

from all_own_funct import roc_auc_ci
try:
    pdf = PdfPages(os.path.join(output_folder,f"Figures_all_t.pdf"))
except PermissionError:
    os.remove(os.path.join(output_folder,f"Figures_all.pdf"))
    os.remove(os.path.join(output_folder,f"Result_scores_all.txt"))
clf = best_random 

y_pred_clas=clf.predict(X_TEST)
# Predict the probabilities, function depends on used classifier

try:
    y_pred_prob=clf.predict_proba(X_TEST)
    y_pred_prob=y_pred_prob[:,1]
except:
    try:
        y_pred_prob=clf.decision_function(X_TEST)
    except:
        y_pred_prob=y_pred_clas

report=classification_report(Y_TEST,y_pred_clas,target_names=['No Reintubation','Reintubation'])
score=clf.score(X_TEST,Y_TEST)
average_precision = average_precision_score(Y_TEST, y_pred_prob)
f1_s=f1_score(Y_TEST, y_pred_clas)

# write scoring metrics to file
with open(os.path.join(output_folder,f"Result_scores_all.txt"),'a') as file:
    file.write(f"Results for RF on training set\n\n")
    file.write(f"Classification report \n {report} \n")
    file.write(f"Hold_out_scores {score} \n")
    file.write(f"Average precision score {average_precision} \n")
    file.write(f"F1 score {f1_s} \n\n\n")

plot_confusion_matrix(clf,X_TEST,Y_TEST)
plt.title(f"Confusion matrix of random forrest")
fig=plt.gcf()
pdf.savefig(fig)
plt.close(fig)

fpr, tpr, _ = roc_curve(Y_TEST,  y_pred_prob)
auc = roc_auc_score(Y_TEST, y_pred_prob)

n_bootstraps = 2000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    # bootstrap by sampling with replacement on the prediction indices
    indices = rng.randint(0, len(y_pred_prob), len(y_pred_prob))
    if len(np.unique(Y_TEST[indices])) < 2:
        # We need at least one positive and one negative sample for ROC AUC
        # to be defined: reject the sample
        continue

    score = roc_auc_score(Y_TEST[indices], y_pred_prob[indices])
    bootstrapped_scores.append(score)
    #print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

# Computing the lower and upper bound of the 90% confidence interval
# You can change the bounds percentiles to 0.025 and 0.975 to get
# a 95% confidence interval instead.
confidence_lower = sorted_scores[int(0.05 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.95 * len(sorted_scores))]
print("Confidence interval for the score: [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))
a=roc_auc_ci(Y_TEST,y_pred_prob)
print(a)


plt.plot(fpr,tpr,label=f"auc={auc}",linewidth=1.5,markersize=1)

plt.legend(loc=4,fontsize='xx-small')
plt.title(f'ROC of Random Forrest hour data')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
axes = plt.gca()
axes.set_xlim([0,1])
axes.set_ylim([0,1])
fig=plt.gcf()
pdf.savefig(fig)
plt.close(fig)


In [ ]:
# Number of trees in random forest
# Number of features to consider at every split
solver =['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
# Maximum number of levels in tree
class_weight=[{0:0.1,1:0.9},'balanced',None]
# Minimum number of samples required to split a node
penalty=['l1', 'l2', 'elasticnet', None]
# Minimum number of samples required at each leaf node
C=np.logspace(-3,3,7)
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'solver':solver,
                'class_weight':class_weight,
                'penalty':penalty,
                'C':C,
                'penalty':penalty}

In [ ]:
# First create the base model to tune
lr = LogisticRegression(max_iter=1000)
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
lr_random = RandomizedSearchCV(estimator = lr , param_distributions = random_grid, n_iter = 20, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
lr_random.fit(X_TRAIN, Y_TRAIN)

In [ ]:
base_model_lr = LogisticRegression(max_iter=1000)
base_model_lr.fit(X_TRAIN, Y_TRAIN)
base_accuracy_lr = evaluate(base_model, X_TEST, Y_TEST,'base LR',output_folder)

best_random_lr = lr_random.best_estimator_
random_accuracy_lr = evaluate(best_random, X_TEST,Y_TEST,'Best Random LR',output_folder)
if base_accuracy_lr > random_accuracy_lr:
    best_random_lr = base_model_lr

In [ ]:
clf = best_random_lr

y_pred_clas=clf.predict(X_TEST)
# Predict the probabilities, function depends on used classifier

try:
    y_pred_prob=clf.predict_proba(X_TEST)
    y_pred_prob=y_pred_prob[:,1]
except:
    try:
        y_pred_prob=clf.decision_function(X_TEST)
    except:
        y_pred_prob=y_pred_clas

report=classification_report(Y_TEST,y_pred_clas,target_names=['No Reintubation','Reintubation'])
score=clf.score(X_TEST,Y_TEST)
average_precision = average_precision_score(Y_TEST, y_pred_prob)
f1_s=f1_score(Y_TEST, y_pred_clas)

with open(os.path.join(output_folder,f"Result_scores_all.txt"),'a') as file:
    file.write(f"Results for LR on training\n\n")
    file.write(f"Classification report \n {report} \n")
    file.write(f"Hold_out_scores {score} \n")
    file.write(f"Average precision score {average_precision} \n")
    file.write(f"F1 score {f1_s} \n\n\n")

plot_confusion_matrix(clf,X_TEST,Y_TEST)
plt.title(f"Confusion matrix of logistic regression")
fig=plt.gcf()
pdf.savefig(fig)
plt.close(fig)

fpr, tpr, _ = roc_curve(Y_TEST,  y_pred_prob)
auc = roc_auc_score(Y_TEST, y_pred_prob)
print(auc)

n_bootstraps = 2000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    # bootstrap by sampling with replacement on the prediction indices
    indices = rng.randint(0, len(y_pred_prob), len(y_pred_prob))
    if len(np.unique(Y_TEST[indices])) < 2:
        # We need at least one positive and one negative sample for ROC AUC
        # to be defined: reject the sample
        continue

    score = roc_auc_score(Y_TEST[indices], y_pred_prob[indices])
    bootstrapped_scores.append(score)
    #print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

# Computing the lower and upper bound of the 90% confidence interval
# You can change the bounds percentiles to 0.025 and 0.975 to get
# a 95% confidence interval instead.
confidence_lower = sorted_scores[int(0.05 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.95 * len(sorted_scores))]
print("Confidence interval for the score: [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))

a=roc_auc_ci(Y_TEST,y_pred_prob)
print(a)

plt.plot(fpr,tpr,label=f"auc={auc}",linewidth=1.5,markersize=1)


plt.legend(loc=4,fontsize='xx-small')
plt.title(f'ROC of Logistic Regression data')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
axes = plt.gca()
axes.set_xlim([0,1])
axes.set_ylim([0,1])
fig=plt.gcf()
pdf.savefig(fig)
plt.close(fig)
pdf.close()


In [ ]:
import pickle
f = open(os.path.join(output_folder,'ran_for.sav'), 'wb')
pickle.dump(best_random, f)
f.close()

f = open(os.path.join(output_folder,'log_reg.sav'), 'wb')
pickle.dump(best_random_lr, f)
f.close()